In [1]:
import sys
sys.path.append('/Users/shintarou/coding/topquartile')

from topquartile.modules.datamodule.dataloader import DataLoader


In [2]:
# Run only if you have errors loading the topquartile module

from pathlib import Path
import sys

root = Path().resolve().parent.parent.parent
sys.path.append(root)

In [3]:
print(root)
print(sys.path)

/Users/shintarou/coding/topquartile
['/opt/homebrew/Caskroom/miniconda/base/envs/topq/lib/python313.zip', '/opt/homebrew/Caskroom/miniconda/base/envs/topq/lib/python3.13', '/opt/homebrew/Caskroom/miniconda/base/envs/topq/lib/python3.13/lib-dynload', '', '/opt/homebrew/Caskroom/miniconda/base/envs/topq/lib/python3.13/site-packages', '/Users/shintarou/coding/topquartile', '/opt/homebrew/Caskroom/miniconda/base/envs/topq/lib/python3.13/site-packages/setuptools/_vendor', PosixPath('/Users/shintarou/coding/topquartile')]


In [4]:
from topquartile.modules.datamodule.dataloader import DataLoader
from topquartile.modules.datamodule.transforms.covariate import (TechnicalCovariateTransform, FundamentalCovariateTransform)
from topquartile.modules.datamodule.transforms.label import BinaryLabelTransform
from topquartile.modules.datamodule.partitions import PurgedTimeSeriesPartition

In [5]:
covtrans_config = [((TechnicalCovariateTransform, dict(sma = [20, 30],
                                                       ema = [20, 30],
                                                       momentum_change=True,
                                                       volatility = [20, 30],)))]

labeltrans_config = [(BinaryLabelTransform, dict(label_duration=20,
                                                quantile=0.75))]

partition_config = dict(n_splits=5, gap=20, max_train_size=504, test_size=60, verbose=False)

In [6]:
dataloader = DataLoader(data_id='dec2024', covariate_transform=covtrans_config,
                  label_transform=labeltrans_config, partition_class=PurgedTimeSeriesPartition,
                  partition_kwargs=partition_config)

In [7]:
folds = dataloader.get_cv_folds()

Data not yet processed. Processing now...
Reading data from: /Users/shintarou/coding/topquartile/topquartile/data/dec2024.csv
Found 342 raw ticker names.
 Applying TechnicalCovariateTransform with params {'sma': [20, 30], 'ema': [20, 30], 'momentum_change': True, 'volatility': [20, 30]}
Applying label transformations globally to the dataset (before partitioning).
 Applying BinaryLabelTransform with params {'label_duration': 20, 'quantile': 0.75} (globally)


/Users/shintarou/coding/topquartile/topquartile/modules/datamodule/transforms/label.py:171: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_copy.groupby(level=self.ticker_level_name, group_keys=False)[self.price_column]


Data processing complete.
Partitioning data using PurgedTimeSeriesPartition for 5 splits across 113 tickers.


/Users/shintarou/coding/topquartile/topquartile/modules/datamodule/dataloader.py:364: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data_grouped_by_ticker = self.data.groupby(level="TickerIndex")


Fold 0: Train shape (56952, 39), Test shape (6780, 39)
Fold 1: Train shape (56952, 39), Test shape (6780, 39)
Fold 2: Train shape (56952, 39), Test shape (6780, 39)
Fold 3: Train shape (56952, 39), Test shape (6780, 39)
Fold 4: Train shape (56952, 39), Test shape (6780, 39)
Partitioning complete. Generated 5 CV folds.


In [8]:
train = folds[0][0]
valid = folds[0][1]


In [9]:
train = train.dropna(how='any', inplace=False)
valid = valid.dropna(how='any', inplace=False)

train.drop('ticker', axis=1, inplace=True)
valid.drop('ticker', axis=1, inplace=True)

to_remove = ['INDEX_RETURN', 'EXCESS_RETURN', '20d_stock_return', 'label']

/var/folders/3h/3r1g6ccn01v_37rgl_0q98rw0000gn/T/ipykernel_89591/3745639844.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('ticker', axis=1, inplace=True)
/var/folders/3h/3r1g6ccn01v_37rgl_0q98rw0000gn/T/ipykernel_89591/3745639844.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid.drop('ticker', axis=1, inplace=True)


In [10]:
train_label = train['EXCESS_RETURN']
train_covariates = train.drop(to_remove, axis=1, inplace=False)

In [11]:
valid_label = valid['EXCESS_RETURN']
valid_covariates = valid.drop(to_remove, axis=1, inplace=False)

In [12]:
from quantile_forest import RandomForestQuantileRegressor
qrf = RandomForestQuantileRegressor()
qrf.fit(train_covariates.to_numpy(), train_label.to_numpy())
y_pred = qrf.predict(valid_covariates.to_numpy(), quantiles=[0.025, 0.5, 0.975])

In [13]:
y_pred

array([[ -3.7660001 ,   4.89947832,  17.77160848],
       [ -5.22030797,   5.71200268,  49.92450134],
       [ -2.57497193,   4.89947832,  18.55850893],
       ...,
       [-13.34686998,   0.19225049,  24.32384295],
       [-24.63845489,   1.2387457 ,  49.32271189],
       [-24.63845489,   1.2387457 ,  49.32271189]], shape=(4940, 3))